<a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>


# CreditCard Furad 실습

In [10]:
import warnings
warnings.filterwarnings('ignore')

df로 환원

In [1]:
import gdown
import os
import pandas as pd
from sklearn.model_selection import train_test_split
# https://drive.google.com/file/d/1TwKDZ24Gp76MhZFP4kRee2uAJYtYZLve/view?usp=sharing
def get_creditcard_dataset():

    google_path = 'https://drive.google.com/uc?id='
    file_id_train = '1TwKDZ24Gp76MhZFP4kRee2uAJYtYZLve'

    gdown.download(google_path+file_id_train,'creditcard.csv',quiet=False)
   
    
    X = pd.read_csv('creditcard.csv')
    y = X.iloc[:,[-1]]
    X = X.iloc[:,:-1]
    
    os.remove('creditcard.csv')

    
    return X, y

X, y = get_creditcard_dataset()

Downloading...
From: https://drive.google.com/uc?id=1TwKDZ24Gp76MhZFP4kRee2uAJYtYZLve
To: C:\Users\N\OneDrive\WorkJ\Machine Learning\Machine_Learning\creditcard.csv
151MB [00:04, 33.1MB/s] 


In [2]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score

def plot_conf_mat(conf_mat):
    import matplotlib.pyplot as plt
    
    fig, ax = plt.subplots(figsize=(2.5, 2.5))
    ax.matshow(conf_mat, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(conf_mat.shape[0]):
        for j in range(conf_mat.shape[1]):
            ax.text(x=j, y=i, s=conf_mat[i, j], va='center', ha='center',fontsize=19)
    
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.tight_layout()
    plt.show()

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
#     print('오차 행렬')
#     plot_conf_mat(confusion)
#     print('정확도:{0:.4f}, 정밀도:{1:.4f}, 재현율:{2:.4f}, F1_score:{3:.4f},\
#             AUC: {4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))
    
    return accuracy, precision, recall, f1, roc_auc

In [3]:
def get_model_train_eval(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    pred_proba = model.predict_proba(X_test)[:, 1]
    
    model_name = str(type(model)).split('.')[-1][:-2]
    accuracy, precision, recall, f1, roc_auc = get_clf_eval(y_test, pred, pred_proba)
    
    return model_name, accuracy, precision, recall, f1, roc_auc

In [4]:
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

lr_clf = LogisticRegression()
lgbm_model= LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)

In [5]:
def get_train_test_dataset(df_X=None, df_y=None, scaler=None, test_size=0.3, random_state=0):
    df_copy = get_preprocessed_df(df_X, scaler)
    X_train, X_test, y_train, y_test = train_test_split(df_copy, df_y, test_size=test_size, 
                                            random_state=random_state, stratify = df_y)
    
    return X_train, X_test, y_train, y_test
    


In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
import numpy as np

scalers = [StandardScaler(), MinMaxScaler(), MaxAbsScaler(), RobustScaler(), 'log1p']

def get_preprocessed_df(df, scaler):
    df_copy = df.copy()
    if scaler=='log1p':
        df_copy['Amount_S'] = np.log1p(df_copy['Amount'].values.reshape(-1, 1))
    else:
        df_copy['Amount_S'] = scaler.fit_transform(df_copy['Amount'].values.reshape(-1, 1))
    
    df_copy.drop(['Time', 'Amount'], axis=1, inplace=True)
    
    return df_copy

In [7]:
result_df = pd.DataFrame(columns=['name', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
idx=0
for scaler in scalers:
    
    X_train, X_test, y_train, y_test = get_train_test_dataset(X, y,  scaler)
    if scaler == 'log1p':
        scaler_name= 'log1p'
    else:
        scaler_name = str(type(scaler)).split('.')[-1][:-2][:-len('Scaler')]
        
    model_name, accuracy, precision, recall, f1, roc_auc = get_model_train_eval(lr_clf, X_train, y_train, X_test, y_test)
    result_df.loc[idx] = str(model_name + scaler_name), accuracy, precision, recall, f1, roc_auc
    idx +=1
    
    model_name, accuracy, precision, recall, f1, roc_auc = get_model_train_eval(lgbm_model, X_train, y_train, X_test, y_test)
    result_df.loc[idx] = str(model_name + scaler_name), accuracy, precision, recall, f1, roc_auc
    idx +=1

C:\Users\N\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\N\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\N\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\N\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:

In [8]:
result_df

,name,accuracy,precision,recall,f1,roc_auc
0,LogisticRegressionStandard,0.999157,0.865385,0.608108,0.714286,0.970227
1,LGBMClassifierStandard,0.999532,0.965517,0.756757,0.848485,0.978225
2,LogisticRegressionMinMax,0.999181,0.882353,0.608108,0.720000,0.971258
3,LGBMClassifierMinMax,0.999520,0.957265,0.756757,0.845283,0.979034
4,LogisticRegressionMaxAbs,0.999181,0.882353,0.608108,0.720000,0.971258
5,LGBMClassifierMaxAbs,0.999520,0.957265,0.756757,0.845283,0.979034
6,LogisticRegressionRobust,0.999157,0.865385,0.608108,0.714286,0.970216
7,LGBMClassifierRobust,0.999520,0.957265,0.756757,0.845283,0.979108
8,LogisticRegressionlog1p,0.999169,0.881188,0.601351,0.714859,0.972683
9,LGBMClassifierlog1p,0.999520,0.957265,0.756757,0.845283,0.979034


In [9]:
result_df.sort_values(['roc_auc','f1'], ascending=False)

,name,accuracy,precision,recall,f1,roc_auc
7,LGBMClassifierRobust,0.999520,0.957265,0.756757,0.845283,0.979108
3,LGBMClassifierMinMax,0.999520,0.957265,0.756757,0.845283,0.979034
5,LGBMClassifierMaxAbs,0.999520,0.957265,0.756757,0.845283,0.979034
9,LGBMClassifierlog1p,0.999520,0.957265,0.756757,0.845283,0.979034
1,LGBMClassifierStandard,0.999532,0.965517,0.756757,0.848485,0.978225
8,LogisticRegressionlog1p,0.999169,0.881188,0.601351,0.714859,0.972683
2,LogisticRegressionMinMax,0.999181,0.882353,0.608108,0.720000,0.971258
4,LogisticRegressionMaxAbs,0.999181,0.882353,0.608108,0.720000,0.971258
0,LogisticRegressionStandard,0.999157,0.865385,0.608108,0.714286,0.970227
6,LogisticRegressionRobust,0.999157,0.865385,0.608108,0.714286,0.970216


# 출처:

* <a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>

<br>
